In [103]:
import pandas as pd
import numpy as np
import sqlite3

dbConn = sqlite3.connect('./data/websiteData.db')
cur = dbConn.cursor()

In [104]:
dfPerCapita = pd.read_csv('./data/co2_tonsPerCapita_annual.csv', low_memory=False)
dfAnnualEmissions = pd.read_csv('./data/annual_co2_emissions_country.csv', low_memory=False)

## Creating Tables

In [ ]:
cur.execute(f"CREATE TABLE Countries (
                Name TEXT UNIQUE
                        NOT NULL,
                Code TEXT PRIMARY KEY
                        UNIQUE
                        NOT NULL
            );"
)


In [ ]:
cur.execute(f"CREATE TABLE Co2cumulative (
                Name TEXT UNIQUE
                        NOT NULL,
                Code TEXT REFERENCES Countries (Code) 
                        UNIQUE
                        NOT NULL
            );"
)

In [ ]:
cur.execute(f"CREATE TABLE Co2PerCapita (
                Code TEXT REFERENCES Countries (Code) 
                        UNIQUE
                        NOT NULL,
                Name TEXT UNIQUE
                        NOT NULL
            );"
)

## Adding Columns

In [105]:
# adding columns in Co2PerCapita table for each year
firstRow = dfPerCapita.iloc[0]
rowSliced = firstRow.drop(labels=["Country Name", "Code"])
years = rowSliced.index.to_list()

for year in years:
    try:
        cur.execute(f"ALTER TABLE Co2PerCapita ADD y{str(year)} REAL;")
    except:
        print(f"column y{year} is already in table")

In [106]:
# adding columns in Co2Cumulative table for each year
firstRow = dfAnnualEmissions.iloc[0]
rowSliced = firstRow.drop(labels=["Entity", "Code"])
years = rowSliced.index.to_list()

for year in years:
    try:
        cur.execute(f"ALTER TABLE Co2cumulative ADD y{str(year)} REAL;")
    except:
        print(f"column y{year} is already in table")

column y1750 is already in table
column y1751 is already in table
column y1752 is already in table
column y1753 is already in table
column y1754 is already in table
column y1755 is already in table
column y1756 is already in table
column y1757 is already in table
column y1758 is already in table
column y1759 is already in table
column y1760 is already in table
column y1761 is already in table
column y1762 is already in table
column y1763 is already in table
column y1764 is already in table
column y1765 is already in table
column y1766 is already in table
column y1767 is already in table
column y1768 is already in table
column y1769 is already in table
column y1770 is already in table
column y1771 is already in table
column y1772 is already in table
column y1773 is already in table
column y1774 is already in table
column y1775 is already in table
column y1776 is already in table
column y1777 is already in table
column y1778 is already in table
column y1779 is already in table
column y17

## Adding Data to Tables

In [107]:
# Adding the countries and codes from both datasets in to the Countries table

for item in [[dfAnnualEmissions, "Entity"], [dfPerCapita, "Country Name"]]:

    for index, row in item[0].iterrows():

        cur.execute(f'SELECT exists(SELECT 1 FROM Countries WHERE Code = "{row["Code"]}") AS row_exists')
        exists = cur.fetchone()[0]

        if not exists:
            cur.execute(f'INSERT or REPLACE INTO Countries (Name, Code) VALUES ("{row[item[1]]}", "{row["Code"]}");')
            dbConn.commit()

In [108]:
# cur.execute(f'DELETE FROM Co2cumulative') 
# dbConn.commit()

In [109]:
# Adding rows to the Co2cumulative table from the dataset

for index, row in dfAnnualEmissions.iterrows():

    cur.execute(f'SELECT exists(SELECT 1 FROM Co2cumulative WHERE Name = "{row["Entity"]}" AND Code = "{row["Code"]}") AS row_exists')
    exists = cur.fetchone()[0]

    if not exists:
        
        cur.execute(f'INSERT INTO Co2cumulative (Name, Code) VALUES ("{row["Entity"]}", "{row["Code"]}");')
        dbConn.commit()

        yearsRow = row.drop(labels=["Entity", "Code"])
        years = yearsRow.index.to_list()

        for year in years:
            cur.execute(f'UPDATE Co2cumulative SET y{year} = "{row[year]}" WHERE Code = "{row["Code"]}";')
            dbConn.commit()
        
cur.execute(f'DELETE FROM Co2cumulative WHERE Code = "ATA";')
dbConn.commit()

In [110]:
# cur.execute(f'DELETE FROM Co2PerCapita')
# dbConn.commit()

In [111]:
# Adding rows to the Co2PerCapita table from the dataset

for index, row in dfPerCapita.iterrows():

    cur.execute(f'SELECT exists(SELECT 1 FROM Co2PerCapita WHERE Name = "{row["Country Name"]}" AND Code = "{row["Code"]}") AS row_exists')
    exists = cur.fetchone()[0]

    if not exists:
        
        cur.execute(f'INSERT INTO Co2PerCapita (Name, Code) VALUES ("{row["Country Name"]}", "{row["Code"]}");')
        dbConn.commit()

        yearsRow = row.drop(labels=["Country Name", "Code"])
        years = yearsRow.index.to_list()

        for year in years:
            query = f"UPDATE Co2PerCapita SET y{year} = ? WHERE Code = ?;"
            cur.execute(query, (row[year], row['Code']))
            dbConn.commit()

In [112]:
dbConn.close()